In [8]:
import data_smoothing
import find_peaks
import peak_character
import peak_classify

In [3]:
data_nm, data_time, data_z = data_smoothing.load_data('20180418_twogaussian_spectralshfit.txt')

In [4]:
smooth_matx = data_smoothing.earth_smooth_matrix(data_nm, data_z)

/Users/demiliu/miniconda3/lib/python3.6/site-packages/sklearn_contrib_py_earth-0.1.0-py3.6-macosx-10.7-x86_64.egg/pyearth/earth.py:802: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()
/Users/demiliu/miniconda3/lib/python3.6/site-packages/sklearn_contrib_py_earth-0.1.0-py3.6-macosx-10.7-x86_64.egg/pyearth/earth.py:1055: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


## load true peak position

In [6]:
import numpy as np
true_pos = np.genfromtxt('20180418_twogaussian_spectralshfit_x0.txt', delimiter='\t')
peak1_true = true_pos[:, 0]
peak2_true = true_pos[:, 1]

In [9]:
threshold = 0
mindist = 0
idx, height, fwhm = peak_character.peak_matrix(data_nm,np.array(smooth_matx), threshold, mindist)

# convert peak index to nm

In [31]:
idx_np = np.nan_to_num(np.array(idx))

peak1 = np.empty(len(data_time))
peak2 = np.empty_like(peak1)
for i in range(np.shape(idx_np)[0]):
    peak1[i] = data_nm[int(idx_np[:, 0][i])]
    peak2[i] = data_nm[int(idx_np[:, 1][i])]

## calculate relative error

In [35]:
def peakpos_error_array_rel(true_pos, pos_output, thre):
    error = 0
    diff = np.empty_like(true_pos)
    
    for i in range(len(true_pos)):
        diff[i] = abs(true_pos[i] - pos_output[i])
        if diff[i] >= thre:
            error += diff[i]/true_pos[i]
            
    return diff, error

In [36]:
peak1_error = peakpos_error_array_rel(peak1_true, peak1, 3)
peak2_error = peakpos_error_array_rel(peak2_true, peak2, 3)

In [37]:
print (peak1_error[1])
print (peak2_error[1])

0.1134003937706003
32.565384615384595


## classify peak

In [38]:
corrected_output = peak_classify.data_grouping(idx, height, fwhm)

In [39]:
peak_dict = peak_classify.cluster_classifier(idx, corrected_output)

In [40]:
peak_dict

{'peak_0': [], 'peak_1': [Position    395.000000
  Height        0.023764
  Width       282.000000
  Time        249.000000
  Name: 499, dtype: float64]}